# Определение токсичности комментариев 

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель:**

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.72. 

**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

* text - текст комментария;
* toxic - целевой признак (1-комментарий токсичен, 0-нетоксичен)

# Импорт необходимых библиотек

In [1]:
import pandas as pd 
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Изучение данных

In [4]:
# чтение данных
data = pd.read_csv('toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Всего 159571 размещенный коментарий. Пропусков в данных нет. Тип данных клонок соответствует их содержанию.

Посмотрим соотношение положительных и токсичных коментариев.

In [6]:
ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
ratio

8.841344371679229

Количество положительных коментариев почти в 9 раз больше, чем отрицательных. Классы несбалансированы.
Для балансировки классов можно использовать:
* настройки модели, изменение весов
* увеличение выборки
* уменьшение выборки

Что бы не потерять данные проверим как изменится значение метрики, если увеличить выборку и если изменить вес класса в параметрах модели.

# Подготовка текста к векторизации

Напишем функции, которая на вход будет принимать текст, а возвращать лемматизированный и очищенный с помощью регулярных выражений текст.

In [9]:
# Напишем функцию позволяющею подготовить текст для вектотризации
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def data_preparation(text):
    
    "используем регулярные выражения для очистки текста"
        
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    
    text_lemm = [lemmatizer.lemmatize(word) for word in text]
    
    clean_text = " ".join(text_lemm)
       
    return clean_text

data['clean_text'] = data['text'].apply(data_preparation)

In [10]:
data.head()

,Unnamed: 0,text,toxic,clean_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wondered sect...
4,4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


Теперь в нашей таблице появилась колонка `clean_text` с обработанным текстом.

# Разбиение на тестовую и обучающую выборку

Разобьем данные на тестовую и обучающую выборку в соотношении 30:70

In [11]:
target = data['toxic']
features = data['clean_text']

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                           test_size = 0.3,
                                                                           random_state=12345)
print(features_train.shape, features_test.shape, sep='\n')

(111504,)
(47788,)


## Векторизация

### Мешок слов (BOW)

In [13]:
count_vect = CountVectorizer()
# векторизация исходной выборки
features_train_bow = count_vect.fit_transform(features_train)
features_test_bow = count_vect.transform(features_test)

### TFIDF

In [14]:
count_tf_idf = TfidfVectorizer()
# векторизация исходной выборки
features_train_idf = count_tf_idf.fit_transform(features_train)
features_test_idf = count_tf_idf.transform(features_test)

# Несбалансированность классов

Так как в дальнейшем при обучении моделей будем использовать кросс-валидацию, то использование upsampled/downsampled данных будет не корректно. В случае upsampling получается так, что в треин и в валидацию (внутри кросс-валидации) попадают одни и те же объекты. При обучении будем использовать class_weight='balanced'.

In [15]:
pipe_lr = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model', LogisticRegression(random_state=12345, class_weight='balanced'))]) 

In [19]:
# модель логистической регрессии с параметром class_weight='balanced' 
cross_val = cross_val_score(pipe_lr, 
                            features_train, 
                            target_train, 
                            scoring='f1', cv=5, n_jobs=-1).mean()

print(cross_val)

/Users/daivanov/opt/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/daivanov/opt/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

0.7482022499935457


## Выводы

* Значение метрики достигает **0.75** при обучении модели.


# Обучение модели

Обучем следующие модели:
* LogisticRegression
* DecisionTreeClassifier
* GradientBoostingClassifier
* RandomForestClassifie

In [20]:
# модели
clf1 = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model_clf1', RandomForestClassifier(random_state=12345))])

clf2 = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model_clf2', LogisticRegression(random_state=42))])

clf3 = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model_clf3', DecisionTreeClassifier(random_state=42))])



clf4 = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model_clf4', GradientBoostingClassifier(random_state=42))])

In [21]:
# для каждой модели параметры для подбора
param1 = {'model_clf1__n_estimators': range (10, 100),
         'model_clf1__max_depth': range (1,80)
         }

param2 = {'model_clf2__solver': ['liblinear', 'sag','saga','newton-cg']}

param3 = {'model_clf3__max_depth': range (1,80),
         'model_clf3__min_samples_split': range(2,8),
         }


param4 = {'model_clf4__n_estimators' : range(10,100),
         'model_clf4__max_depth' : range(1, 80),
         'model_clf4__learning_rate': np.arange(0.01, 0.2, 0.01),
         }

In [22]:
# обучение моделей с использованием увеличенной выборки
models = [clf1, clf2, clf3, clf4]
params = [param1, param2, param3, param4]
results = pd.DataFrame({'classificator': [],
                        'best_score': [],
                       'upsample': []})

for i in range(len(models)):
    rs = RandomizedSearchCV(models[i], params[i], cv=3, n_jobs=-1, scoring='f1')
    rs.fit(features_train, target_train)
    print(rs.best_params_, rs.best_score_, sep='\n')
    results = results.append({'classificator': models[i],
               'best_score': rs.best_score_,
                'upsample': 'No'}, ignore_index=True)
    
display(results)

{'model_clf1__n_estimators': 10, 'model_clf1__max_depth': 72}
0.29824896304709964
{'model_clf2__solver': 'saga'}
0.7119608149999276
{'model_clf3__min_samples_split': 4, 'model_clf3__max_depth': 78}
0.7229912519470202
{'model_clf4__n_estimators': 29, 'model_clf4__max_depth': 72, 'model_clf4__learning_rate': 0.11}
0.7249977776781972


,classificator,best_score,upsample
0,"(CountVectorizer(), TfidfTransformer(), Random...",0.298249,No
1,"(CountVectorizer(), TfidfTransformer(), Logist...",0.711961,No
2,"(CountVectorizer(), TfidfTransformer(), Decisi...",0.722991,No
3,"(CountVectorizer(), TfidfTransformer(), Gradie...",0.724998,No


Модель показавшую наилучшее значение метрики проверим на тестовой выборке.

# Проверка на тестовой выбоке

In [24]:
pipe_lr = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model', LogisticRegression(random_state=12345, solver = 'newton-cg'))])

pipe_lr.fit(features_train, target_train)
prediction = pipe_lr.predict(features_test)
f1 = f1_score(target_test, prediction)
                             
print(f1)

0.7385303991048116


Посмотрим матрицу ошибок

In [1]:
cm = confusion_matrix(target_test, prediction)
plt.figure(figsize=(5, 3))
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Test Confusion Matrix')
plt.show()

NameError: name 'confusion_matrix' is not defined

# Проверка на адекватность

Используем простейшую модель предсказывающую один класс для проверки нашей модели на адекватность.

In [ ]:
model = DummyClassifier(strategy='constant', constant= 1)
model.fit(features_train_bow, target_train)
prediction = model.predict(features_test_bow)
score = f1_score(target_test, prediction)
print('f1 для простейшей модели - ', score)

Для модели предсказывающей постоянное число значение метрики f1 равно 0.185. Это значительно меньше, чем у модели логистической регрессии. Следовательно выбраная нами и обученная модель адекватна.

# Вывод

В ходе работы наилучшей моделью для классификации комментариев на положительные и токсичные оказалась модель **LogisticRegression**. На тестовой выборке значение метрики f1 составило **0.759**. 

Несбалансированность классов была преодолена с помощью увеличения выборки.
Исходные данные были подготовлены с помощью лемматизации, удаления спец символов и цифр и дальнейшей векторизации ттекста. 